In [14]:
from datasets import load_dataset
print('test')

test


In [55]:
food = load_dataset("food101", split="train[:2500]")

In [56]:
print(len(food))
food = food.train_test_split(test_size=0.2)

2500


In [57]:
#create dictionnary for labels

labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [80]:
#load pretrained model

from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
checkpoint2 = "google/vit-large-patch16-224"
checkpoint3= "google/vit-base-patch32-384"


image_processor = AutoImageProcessor.from_pretrained(checkpoint3)

#image_processor = AutoImageProcessor.from_pretrained(checkpoint)
#image_processor2 = AutoImageProcessor.from_pretrained(checkpoint2)
#image_processor3 = AutoImageProcessor.from_pretrained(checkpoint3)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [81]:
#check augmentations done in this paper: https://www.sciencedirect.com/science/article/pii/S0260877423004314

from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [82]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [83]:
food = food.with_transform(transforms)

In [84]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [85]:
import evaluate

accuracy = evaluate.load("accuracy")

In [86]:
import numpy as np


def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [87]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint3,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch32-384 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([101]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([101, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

import torch
torch.cuda.empty_cache()

training_args = TrainingArguments(
    report_to="wandb",
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()



Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.087389,0.696000
2,4.058300,0.766141,0.856000
3,1.106800,0.278449,0.950000
4,0.283900,0.154613,0.956000
5,0.158900,0.117758,0.970000
6,0.158900,0.109794,0.972000
7,0.096400,0.124353,0.962000
8,0.069100,0.114829,0.970000
9,0.067200,0.109774,0.960000
10,0.056600,0.076504,0.974000


/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.p

TrainOutput(global_step=80, training_loss=0.7371422052383423, metrics={'train_runtime': 1909.3716, 'train_samples_per_second': 10.475, 'train_steps_per_second': 0.042, 'total_flos': 4.65048354226176e+18, 'train_loss': 0.7371422052383423, 'epoch': 10.0})

In [89]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

import torch
torch.cuda.empty_cache()
training_args = TrainingArguments(
    report_to="wandb",
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    warmup_ratio=0,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()



Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.098831,0.976000
2,0.066900,0.119065,0.958000
3,0.037700,0.098044,0.970000
4,0.028400,0.079966,0.972000
5,0.033200,0.096409,0.968000
6,0.033200,0.087677,0.970000
7,0.026300,0.093177,0.974000
8,0.019600,0.101641,0.968000
9,0.026700,0.102387,0.964000
10,0.019200,0.060703,0.978000


/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.p

TrainOutput(global_step=80, training_loss=0.03225158471614122, metrics={'train_runtime': 1925.0973, 'train_samples_per_second': 10.389, 'train_steps_per_second': 0.042, 'total_flos': 4.65048354226176e+18, 'train_loss': 0.03225158471614122, 'epoch': 10.0})

In [68]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__
import torch
torch.cuda.empty_cache()

training_args = TrainingArguments(
    report_to="wandb",
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    warmup_ratio=0.2,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()



Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.095004,0.970000
2,0.002100,0.081320,0.968000
3,0.003800,0.164224,0.962000
4,0.003200,0.090961,0.972000
5,0.012100,0.149852,0.974000
6,0.012100,0.089324,0.970000
7,0.024600,0.106158,0.974000
8,0.023000,0.139028,0.964000
9,0.018200,0.103192,0.970000
10,0.009500,0.083404,0.972000


/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/rohitkumar/.conda/envs/november/lib/python3.10/site-packages/torch/nn/parallel/_functions.p

TrainOutput(global_step=80, training_loss=0.012068350007757545, metrics={'train_runtime': 3982.764, 'train_samples_per_second': 5.022, 'train_steps_per_second': 0.02, 'total_flos': 5.48051657674752e+18, 'train_loss': 0.012068350007757545, 'epoch': 10.0})